This code below is the one used for interacting between our react interface and our model there's something to change in this code it will be commented but let sum up:

*   model name : use your model name and username (huggingface)
*   change origin allowed if it's different
*   give your token from ngrok (free)



In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:
!pip install "unsloth[cu121] @ git+https://github.com/unslothai/unsloth.git"

#**Load your trained and saved model**

In [ ]:
from unsloth import FastLanguageModel
# PUT YOUR USERNAME, NAME THE MODEL
model_name = "username/name_model"
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# **Get our template**

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

#**Get our api ready for inference**


In [ ]:
from fastapi import FastAPI
import pickle
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()



app = FastAPI()
# Add your origin url if it is different
origins = [
    "http://localhost:3000",
    "localhost:3000"
]


app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)
mapping = {
    "role": "from",
    "content": "value",
    "user": "human",
    "assistant": "gpt"
}
@app.get("/")
def index():
    return {"message" : "This is the homepage of the API"}
class Request(BaseModel):
    input : str
@app.post("/predict")
def get_response(req: Request):
  messages = [
    {"from": "human", "value": req.input},]
  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt").to("cuda")
  outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
  response = tokenizer.batch_decode(outputs)
  assistant_reply = response[0][len(req.input)+19+len("<|assistant|>"):-7]
  return {"messages" : assistant_reply}


Put your token from ngrok

In [ ]:
!ngrok authtoken "ur_token"

#**Running the serve**

In [ ]:
from pyngrok import ngrok
port = 8001
ngrok_tunnel = ngrok.connect(port)

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)

import nest_asyncio
nest_asyncio.apply()

# finally run the app
import uvicorn
uvicorn.run(app, port=port)